# ODM 선적 실적 보기

In [1]:
from tool import *
# import plotly.express as px
from openpyxl import Workbook
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.drawing.image import Image
from openpyxl.styles import PatternFill, Alignment, Font, Border, Side

## 4) PC ODM 월별 선적 실적 보기

In [2]:
with open('D:/Data/ODM_SR.db', 'rb') as f:
    sr = pickle.load(f)
with open('D:/Data/sitemap.db', 'rb') as f:
    ste = pickle.load(f)

In [3]:
sr = sr[sr['Division']=='PC'] # PC 실적만 조회
sr = sr.reset_index()
sr['Week Name'] = sr['Ship Date'].map(get_weekname)
sr['To Site'] = sr['To Site'].replace(ship_to_map)
sr = pd.merge(sr, ste[['Region', 'Subsidiary', 'To Site', 'Country']], how='left')
sr[['Year', 'Month']] = sr['Week Name'].apply(lambda x:f'{datetime.date.fromisoformat(x[:10]).isocalendar().year}-{get_month_from_date(datetime.date.fromisoformat(x[:10])):02d}').str.split('-', expand=True)
sr['Series'] = sr['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).str.replace('D|G|N|W|C', '').apply(lambda x:x[:6])
sr = sr[sr['Year']=='2023'] # 2023년 실적으로 조회

C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_11780\731353633.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  sr['Series'] = sr['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).str.replace('D|G|N|W|C', '').apply(lambda x:x[:6])


In [4]:
sr = sr.pivot_table('Ship', index=['From Site', 'Country', 'Series'], columns=['Year', 'Month'], aggfunc=sum).fillna(0)
d = datetime.datetime.today()
filepath_to_save = 'D:/PC_ODM_월별 실적/2023년 PC_ODM_월별 선적 실적_{}.xlsx'.format(d.strftime('%y%m%d'))
sr.to_excel(filepath_to_save, startrow=2)

In [5]:
wb = load_workbook(filepath_to_save)
color = PatternFill(fgColor='9999FF', fill_type='solid') # Blue
big_size = Font(name='맑은 고딕', size=20, bold=True, color='000099')
middle_size = Font(name='맑은 고딕', size=15, bold=True, color='000099')

border_style = Side(border_style="thin", color="000000")
border = Border(top=border_style, right=border_style, bottom=border_style, left=border_style)

In [6]:
sh = wb[wb.sheetnames[0]]
sh['A1'] = f'■ {d.strftime("%y%m%d")} PC ODM Monthly Shipment Report'
sh['A1'].font = middle_size
sh['C2'] = 'Sub TTL'
for i in range(4, sh.max_column+1):
    col_letter = get_column_letter(i)
    sh.cell(row=2, column=i).value = f'=subtotal(9, {col_letter}6:{col_letter}{sh.max_row})'
    sh.cell(row=2, column=i).border = border
min_row_num = 3
height = len(sr.columns[0])
max_row_num = min_row_num + height - 1
for row in sh.iter_rows(min_row=min_row_num, min_col=3, max_row=max_row_num, max_col=sh.max_column):
    for cell in row:
        cell.fill = color
min_row_num = max_row_num + 1
for row in sh.iter_rows(min_row=min_row_num, min_col=1, max_row=min_row_num, max_col=len(sr.index[0])):
    for cell in row:
        cell.fill = color
min_row_num = min_row_num + 1
max_row_num = min_row_num + sr.shape[0] - 1
for row in sh.iter_rows(min_row=min_row_num, min_col=len(sr.index[0]), max_row=max_row_num, max_col=sh.max_column):
    for cell in row:
        cell.border = border
        cell.number_format = '#,##0;[RED]Δ#,##0;"-"'
        
sh.freeze_panes = 'D5'
sh.column_dimensions['A'].width = 20
sh.column_dimensions['B'].width = 10

for name in wb.sheetnames:
    wb[name].sheet_view.showGridLines = False
    
wb.save(filepath_to_save)
wb.close()